In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import sklearn.metrics

# Description: Predicting the occurrence of seismic events based on geological information from mines
# This is a binary classification problem, where 1 (True) means that a seismic bump occurs in the next shift
# 0 (False) means that a seismic bump will NOT occur in the next shift
# Dataset Source: https://archive.ics.uci.edu/ml/datasets/seismic-bumps

randomSeed = 50

In [2]:
# Columns from data specification provided by dataset authors
columns = ["seismic", "seismoacoustic", "shift", "genergy", "gpuls", "gdenergy", "gdpuls", "ghazard", "nbumps",
          "nbumps2", "nbumps3", "nbumps4", "nbumps5", "nbumps6", "nbumps7", "nbumps89", "energy", "maxenergy", "class"]
# File extension is not .csv, but by removing the comments in the file, it can be imported as a .csv file
data = pd.read_csv("seismic-bumps.arff", names=columns)

In [3]:
print(data)
# There are four non numeric columns that must be converted to a numeric format
print(data.info())
# There is a large difference in the range of values between the columns, so they should be placed on the same scale
# for a better performing algorithm
# Some of the columns also appear to have all 0 values. These columns should be removed as they appear to provide no
# useful information
print(data.describe())

     seismic seismoacoustic shift  genergy  gpuls  gdenergy  gdpuls ghazard  \
0          a              a     N    15180     48       -72     -72       a   
1          a              a     N    14720     33       -70     -79       a   
2          a              a     N     8050     30       -81     -78       a   
3          a              a     N    28820    171       -23      40       a   
4          a              a     N    12640     57       -63     -52       a   
...      ...            ...   ...      ...    ...       ...     ...     ...   
2579       b              a     W    81410    785       432     151       b   
2580       b              a     W    42110    555       213     118       a   
2581       b              a     W    26960    540       101     112       a   
2582       a              a     W    16130    322         2       2       a   
2583       a              a     W    12750    235       -10     -10       a   

      nbumps  nbumps2  nbumps3  nbumps4  nbumps5  n

In [4]:
# The columns with all 0 values have no correlation value, so this shows that they can be safely removed
print(data.corr())

            genergy     gpuls  gdenergy    gdpuls    nbumps   nbumps2  \
genergy    1.000000  0.748020  0.048514  0.071554  0.220720  0.143587   
gpuls      0.748020  1.000000  0.293036  0.382906  0.300923  0.207390   
gdenergy   0.048514  0.293036  1.000000  0.811944  0.030039  0.041246   
gdpuls     0.071554  0.382906  0.811944  1.000000  0.057996  0.051106   
nbumps     0.220720  0.300923  0.030039  0.057996  1.000000  0.804978   
nbumps2    0.143587  0.207390  0.041246  0.051106  0.804978  1.000000   
nbumps3    0.191753  0.225695 -0.012189  0.014735  0.803364  0.350724   
nbumps4    0.150589  0.256547  0.036916  0.066195  0.395052  0.161296   
nbumps5   -0.009863  0.049450  0.123229  0.141044  0.069613 -0.005251   
nbumps6         NaN       NaN       NaN       NaN       NaN       NaN   
nbumps7         NaN       NaN       NaN       NaN       NaN       NaN   
nbumps89        NaN       NaN       NaN       NaN       NaN       NaN   
energy     0.080828  0.187350  0.105971  0.143277  

In [5]:
# Remove useless columns
data = data.drop(labels=["nbumps6", "nbumps7", "nbumps89"], axis=1)
print(data)

     seismic seismoacoustic shift  genergy  gpuls  gdenergy  gdpuls ghazard  \
0          a              a     N    15180     48       -72     -72       a   
1          a              a     N    14720     33       -70     -79       a   
2          a              a     N     8050     30       -81     -78       a   
3          a              a     N    28820    171       -23      40       a   
4          a              a     N    12640     57       -63     -52       a   
...      ...            ...   ...      ...    ...       ...     ...     ...   
2579       b              a     W    81410    785       432     151       b   
2580       b              a     W    42110    555       213     118       a   
2581       b              a     W    26960    540       101     112       a   
2582       a              a     W    16130    322         2       2       a   
2583       a              a     W    12750    235       -10     -10       a   

      nbumps  nbumps2  nbumps3  nbumps4  nbumps5  e

In [6]:
# Separate the data into features and labels
X = data.iloc[:, 0:-1]
y = data["class"]
print("X: ", X)
print("Y: ", y)

X:       seismic seismoacoustic shift  genergy  gpuls  gdenergy  gdpuls ghazard  \
0          a              a     N    15180     48       -72     -72       a   
1          a              a     N    14720     33       -70     -79       a   
2          a              a     N     8050     30       -81     -78       a   
3          a              a     N    28820    171       -23      40       a   
4          a              a     N    12640     57       -63     -52       a   
...      ...            ...   ...      ...    ...       ...     ...     ...   
2579       b              a     W    81410    785       432     151       b   
2580       b              a     W    42110    555       213     118       a   
2581       b              a     W    26960    540       101     112       a   
2582       a              a     W    16130    322         2       2       a   
2583       a              a     W    12750    235       -10     -10       a   

      nbumps  nbumps2  nbumps3  nbumps4  nbumps

In [7]:
from sklearn.preprocessing import LabelEncoder
# The features seismic, seismoacousatic, shift and ghzard are categorical data consisting of characters. 
# They must be converted to numeric values
X["seismic"] = LabelEncoder().fit_transform(X["seismic"])
X["seismoacoustic"] = LabelEncoder().fit_transform(X["seismoacoustic"])
X["shift"] = LabelEncoder().fit_transform(X["shift"])
X["ghazard"] = LabelEncoder().fit_transform(X["ghazard"])
print(X)

      seismic  seismoacoustic  shift  genergy  gpuls  gdenergy  gdpuls  \
0           0               0      0    15180     48       -72     -72   
1           0               0      0    14720     33       -70     -79   
2           0               0      0     8050     30       -81     -78   
3           0               0      0    28820    171       -23      40   
4           0               0      0    12640     57       -63     -52   
...       ...             ...    ...      ...    ...       ...     ...   
2579        1               0      1    81410    785       432     151   
2580        1               0      1    42110    555       213     118   
2581        1               0      1    26960    540       101     112   
2582        0               0      1    16130    322         2       2   
2583        0               0      1    12750    235       -10     -10   

      ghazard  nbumps  nbumps2  nbumps3  nbumps4  nbumps5  energy  maxenergy  
0           0       0        0  

In [8]:
# Scaling numeric data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Need to turn back into a DataFrame, or preserve the existing DataFrame structure
print(X)

[[-0.73230209 -0.77142023 -1.34374329 ... -0.06830542 -0.24332671
  -0.22108685]
 [-0.73230209 -0.77142023 -1.34374329 ... -0.06830542 -0.14551225
  -0.11774749]
 [-0.73230209 -0.77142023 -1.34374329 ... -0.06830542 -0.24332671
  -0.22108685]
 ...
 [ 1.36555667 -0.77142023  0.74418976 ... -0.06830542 -0.24332671
  -0.22108685]
 [-0.73230209 -0.77142023  0.74418976 ... -0.06830542 -0.24332671
  -0.22108685]
 [-0.73230209 -0.77142023  0.74418976 ... -0.06830542 -0.24332671
  -0.22108685]]


In [10]:
#Setting up splits
from sklearn.model_selection import train_test_split
#I wasn't sure if we would want the partitions randomized, so I made a randomized and unrandomized verson.  
#If we don't end up needing the unrandomized version, I can easily take it out, but I thought I'd put it there just in case.
# From Jacob: There's no disadvantage to shuffling, so I removed the unshuffled version
trainX, testX, trainY, testY = train_test_split(X, y, random_state=randomSeed)

#I printed the shape as well to confirm it is working properly since the arrays are a bit hard to interpret at the moment

print(trainX)
print(trainX.shape)
print(testX)
print(testX.shape)
print(trainY)
print(testY)

[[-0.73230209 -0.77142023 -1.34374329 ... -0.06830542 -0.11616791
  -0.11774749]
 [-0.73230209 -0.77142023 -1.34374329 ... -0.06830542 -0.24332671
  -0.22108685]
 [-0.73230209 -0.77142023  0.74418976 ... -0.06830542  0.23596412
   0.24394029]
 ...
 [-0.73230209 -0.77142023 -1.34374329 ... -0.06830542 -0.24332671
  -0.22108685]
 [ 1.36555667 -0.77142023  0.74418976 ... -0.06830542 -0.24332671
  -0.22108685]
 [-0.73230209  1.12339905  0.74418976 ... -0.06830542 -0.24332671
  -0.22108685]]
(1938, 15)
[[-0.73230209 -0.77142023  0.74418976 ... -0.06830542 -0.24332671
  -0.22108685]
 [-0.73230209 -0.77142023  0.74418976 ... -0.06830542 -0.24332671
  -0.22108685]
 [ 1.36555667  1.12339905  0.74418976 ... -0.06830542 -0.22865454
  -0.20558595]
 ...
 [-0.73230209  3.01821832  0.74418976 ... -0.06830542 -0.24332671
  -0.22108685]
 [-0.73230209 -0.77142023  0.74418976 ... -0.06830542 -0.24332671
  -0.22108685]
 [ 1.36555667 -0.77142023  0.74418976 ... -0.06830542 -0.03791635
  -0.11774749]]
(646,

In [11]:
logClassifier = LogisticRegression(random_state=randomSeed).fit(trainX, trainY)
logScores = cross_val_score(logClassifier, testX, testY)
print(logScores)
logPred = logClassifier.predict(testX)
# The accuracy is very high, but this is due to the unbalanced data set and complexity of the data set
# Precision and recall are both 0 because the classifier was unable to correctly classify any positive instances
print(sklearn.metrics.accuracy_score(testY, logPred))
print(sklearn.metrics.precision_score(testY, logPred, average='binary', pos_label=1))
print(sklearn.metrics.recall_score(testY, logPred, average='binary', pos_label=1))

[0.91538462 0.93023256 0.93023256 0.89147287 0.92248062]
0.9195046439628483
0.0
0.0


In [12]:
# Even a random forest classifier has the same difficulty
randomForestClassifier = RandomForestClassifier(random_state=randomSeed).fit(trainX, trainY)
randomForestScores = cross_val_score(randomForestClassifier, testX, testY)
print(randomForestScores)
randomForestPred = randomForestClassifier.predict(testX)
print(sklearn.metrics.accuracy_score(testY, randomForestPred))
print(sklearn.metrics.precision_score(testY, logPred, average='binary', pos_label=1))
print(sklearn.metrics.recall_score(testY, logPred, average='binary', pos_label=1))

[0.92307692 0.9379845  0.93023256 0.92248062 0.91472868]
0.9272445820433437
0.0
0.0


In [13]:
# How unbalanced is the dataset?
posCount = (y == 1).sum()
negCount = (y == 0).sum()
print("Positive instances: ", posCount)
print("Negative instances: ", negCount)
print("Positive Percentage: ", posCount / negCount)
print("The dataset is very unbalanced - only 7% of the classes are positive. This has to be resolved.");

Positive instances:  170
Negative instances:  2414
Positive Percentage:  0.07042253521126761
The dataset is very unbalanced - only 7% of the classes are positive. This has to be resolved.


In [20]:
#I think this helped.  The accuracy isnt as good but its actually predicting now.
#the accuracy didnt tank quite as much as I thought it would.
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
clfSVC = SVC(kernel='linear', class_weight='balanced', probability=True)
clfSVC.fit(trainX, trainY)
predSVC = clfSVC.predict(testX)
print(predSVC)
print( accuracy_score(testY, predSVC) )
SVCScores = cross_val_score(clfSVC, testX, testY)
print(SVCScores)

[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0
 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 0 0 1 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0
 1 1 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1
 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0
 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0
 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 1 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0
 0 0 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1
 0 0 1 0 0 1 0 0 1 0 0 0 